# Code 1

The script is used to load infrastructure information (including turnouts, joints, and bridges) and plot them on the map.

In [ ]:
# Install Environement for Google Colab Virtual Machine
!pip install -U kaleido
!plotly_get_chrome

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio # Import plotly.io for saving images

from google.colab import userdata

# Define file paths
files = {
    "Bridge": userdata.get('Data_converted_coordinates_Resultat_Bridge'),
    "RailJoint": userdata.get('Data_converted_coordinates_Resultat_RailJoint'),
    "Turnout": userdata.get('Data_converted_coordinates_Turnout')
}

# Define marker styles with different colors and sizes
marker_styles = {
    "Bridge": {"color": "red", "size": 10},
    "RailJoint": {"color": "blue", "size": 8},
    "Turnout": {"color": "green", "size": 12}
}

# Load data

from google.colab import drive
drive.mount('/content/drive')

data_frames = []
for category, file in files.items():
    try:
        df = pd.read_csv(file, encoding="utf-8")  # Load CSV with UTF-8 encoding
        df.columns = df.columns.str.strip()  # Strip column names of extra spaces
        if "Latitude" in df.columns and "Longitude" in df.columns:
            df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")  # Convert Latitude to numeric
            df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")  # Convert Longitude to numeric
            df = df[["Latitude", "Longitude"]]  # Select necessary columns
            df["Category"] = category  # Add category column
            data_frames.append(df)
            print(f"Successfully loaded {category} data: {len(df)} rows")
        else:
            print(f"Warning: {category} file does not contain 'Latitude' and 'Longitude' columns.")
            print(f"Available columns: {df.columns.tolist()}")
    except Exception as e:
        print(f"Error loading {category}: {e}")

# Combine all data
if data_frames:
    data = pd.concat(data_frames, ignore_index=True)
else:
    raise ValueError("No valid data found. Check your CSV files.")

# Debugging: Check if all categories exist
print("Data counts per category:\n", data["Category"].value_counts())

# Check if latitude and longitude values are valid
print("Data summary:\n", data.describe())

# Check for missing values in the data
print("Missing values per column:\n", data.isnull().sum())

# Drop rows with missing Latitude or Longitude values (if any)
data = data.dropna(subset=["Latitude", "Longitude"])

# Add additional debugging to check data before plotting
for category in marker_styles.keys():
    category_data = data[data["Category"] == category]
    print(f"{category}: {len(category_data)} rows")
    if len(category_data) > 0:
        print(f"Sample coordinates for {category}:")
        print(category_data[["Latitude", "Longitude"]].head(2))
    else:
        print(f"WARNING: No data for {category}!")

# Create a Plotly map with custom size (width x height in pixels)
fig = go.Figure()

# Add each category as a separate trace
for category, style in marker_styles.items():
    category_data = data[data["Category"] == category]

    if len(category_data) > 0:
        fig.add_trace(go.Scattermapbox(
            lat=category_data["Latitude"],
            lon=category_data["Longitude"],
            mode="markers",
            marker=dict(
                color=style["color"],
                size=style["size"]
            ),
            name=category
        ))
    else:
        print(f"Skipping {category} - no data available")

# Update layout for the map with custom size
fig.update_layout(
    mapbox_style="open-street-map",
    title="Railway Map with Bridges, Joints, and Turnouts",
    legend_title_text="Legend",
    width=1200,  # Set the width of the figure in pixels
    height=800,  # Set the height of the figure in pixels
    margin={"r": 0, "t": 50, "l": 50, "b": 0},
    mapbox=dict(
        zoom=10,
        center=dict(lat=data["Latitude"].mean(), lon=data["Longitude"].mean())  # Center map around the data
    )
)

# Check if we have any traces
if len(fig.data) == 0:
    print("WARNING: No valid traces were added to the figure. Check your data!")

# Show the figure
fig.show()

# Save the figure as a static image to Google Drive
output_path = '/content/drive/My Drive/railway_map.png' # Define the output path in your Google Drive
try:
    pio.write_image(fig, output_path)
    print(f"Map saved successfully to {output_path}")
except Exception as e:
    print(f"Error saving map image: {e}")


Plotly will install a copy of Google Chrome to be used for generating static images of plots.
Chrome will be installed at: /usr/local/lib/python3.11/dist-packages/choreographer/cli/browser_exe
Do you want to proceed? [y/n] y
Installing Chrome for Plotly...
Chrome installed successfully.
The Chrome executable is now located at: /usr/local/lib/python3.11/dist-packages/choreographer/cli/browser_exe/chrome-linux64/chrome
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully loaded Bridge data: 25 rows
Successfully loaded RailJoint data: 20 rows
Successfully loaded Turnout data: 75 rows
Data counts per category:
 Category
Turnout      75
Bridge       25
RailJoint    20
Name: count, dtype: int64
Data summary:
          Latitude   Longitude
count  120.000000  120.000000
mean    60.792431   14.921160
std      0.191516    0.284255
min     60.510878   14.518605
25%     60.587961   14.570193
50%     60.748120

/tmp/ipython-input-4246143758.py:84: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



Map saved successfully to /content/drive/My Drive/railway_map.png


# Code 2

The script is an end-to-end prototype for visualizing vehicle telemetry data. It guides users to load five CSV files (latitude, longitude, two vibration channels, and speed), converts them into Pandas DataFrames with synthetic timestamps, and combines relevant columns into GPS and vibration tables. The vibration data is segmented into 10-second windows (at 500Hz). Using Plotly and Dash, it builds an interactive dashboard: the left panel displays GPS points on an OpenStreetMap base, and the right panel shows vibration data. When a user clicks a map point, the dashboard fetches and plots the related vibration segment, allowing users to link spatial location with corresponding vibration measurements for easy inspection.

In [ ]:
'''
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html, Input, Output
import tkinter as tk
from tkinter import filedialog

# ====================
# Data Loading using Tkinter
# ====================
root = tk.Tk()
root.withdraw()  # Hide the root Tk window

# Define file keys for CSV files.
files = {
    "latitude": None,
    "longitude": None,
    "vibration1": None,
    "vibration2": None,
    "speed": None
}

def load_file(key):
    file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if file_path:
        files[key] = file_path
        print(f"{key.capitalize()} file loaded: {file_path}")

print("Select Latitude File")
load_file("latitude")
print("Select Longitude File")
load_file("longitude")
print("Select Vibration 1 File")
load_file("vibration1")
print("Select Vibration 2 File")
load_file("vibration2")
print("Select Speed File")
load_file("speed")

# Load each CSV into a DataFrame and add a 'timestamp' using the row index.
dataframes = {}
for key, file_path in files.items():
    if file_path:
        df = pd.read_csv(file_path, header=None, names=[key])
        df['timestamp'] = df.index
        dataframes[key] = df
    else:
        print(f"{key.capitalize()} file not selected.")

# ====================
# Create GPS DataFrame by merging latitude and longitude.
# ====================
if "latitude" in dataframes and "longitude" in dataframes:
    df_gps = pd.merge(dataframes["latitude"], dataframes["longitude"], on="timestamp")
    # Rename columns for consistency
    df_gps = df_gps.rename(columns={"latitude": "Latitude", "longitude": "Longitude"})
    # Add an index column for use in the interactive plot
    df_gps["PointIndex"] = df_gps.index
else:
    print("Latitude or Longitude data is missing.")
    df_gps = pd.DataFrame(columns=["Latitude", "Longitude", "PointIndex"])

# ====================
# Merge the two vibration signals on 'timestamp'
# ====================
if "vibration1" in dataframes and "vibration2" in dataframes:
    df_vibration_merged = pd.merge(
        dataframes["vibration1"],
        dataframes["vibration2"],
        on="timestamp"
        # When the column names differ (here: vibration1 vs vibration2), suffixes are not needed.
    )
    # You may rename columns if desired; here they remain "vibration1" and "vibration2"
else:
    print("Vibration data files are missing.")
    df_vibration_merged = pd.DataFrame()

# ====================
# Data Preprocessing and Segmentation for Vibration Data
# ====================
dt_vibration = 0.002  # seconds per sample (e.g. 500 Hz sampling rate)
segment_duration_seconds = 10
segment_length = int(segment_duration_seconds / dt_vibration)
if not df_vibration_merged.empty:
    num_segments = len(df_vibration_merged) // segment_length
    segments = []
    for i in range(num_segments):
        seg = df_vibration_merged.iloc[i * segment_length: (i + 1) * segment_length][["vibration1", "vibration2"]].values
        segments.append(seg)
    segments = np.array(segments)
    print("Segmented vibration data shape:", segments.shape)
else:
    segments = np.array([])
    print("No vibration data available for segmentation.")

# ====================
# Build the Interactive Dash App
# ====================

# Create the interactive GPS map using Plotly Express.
if not df_gps.empty:
    # Use custom_data to store the point index so that it will be available in callbacks.
    map_fig = px.scatter_mapbox(
        df_gps,
        lat="Latitude",
        lon="Longitude",
        custom_data=["PointIndex"],
        zoom=10,
        title="GPS Points with Vibration Data"
    )
    map_fig.update_layout(mapbox_style="open-street-map", height=600)
else:
    map_fig = go.Figure()
    map_fig.update_layout(title="No GPS Data Available", height=600)

# Create an initial empty vibration plot figure.
vib_empty_fig = go.Figure()
vib_empty_fig.update_layout(
    title="Vibration Signal",
    xaxis_title="Time (s)",
    yaxis_title="Acceleration"
)

# Initialize Dash app.
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        dcc.Graph(id="gps-map", figure=map_fig)
    ], style={'width': '48%', 'display': 'inline-block', 'vertical-align': 'top'}),
    html.Div([
        dcc.Graph(id="vibration-plot", figure=vib_empty_fig)
    ], style={'width': '48%', 'display': 'inline-block', 'vertical-align': 'top'})
])

# --------------------
# Callback to Update the Vibration Plot Based on Clicked GPS Point
# --------------------
@app.callback(
    Output('vibration-plot', 'figure'),
    [Input('gps-map', 'clickData')]
)
def update_vibration_plot(clickData):
    # If no point is selected, return the empty vibration figure.
    if clickData is None:
        return vib_empty_fig

    # Retrieve the selected GPS point index from custom_data.
    point_index = clickData['points'][0]['customdata'][0]

    if segments.size == 0:
        empty_fig = go.Figure()
        empty_fig.update_layout(
            title="No Vibration Data Available",
            xaxis_title="Time (s)",
            yaxis_title="Acceleration"
        )
        return empty_fig

    # Map the GPS point to a vibration segment. If the selected index exceeds
    # the available number of segments, use the last segment as fallback.
    if point_index < segments.shape[0]:
        selected_segment = segments[point_index]
    else:
        selected_segment = segments[-1]

    # Create a time axis for the selected segment.
    time_axis = np.arange(segment_length) * dt_vibration

    vib_fig = go.Figure()
    vib_fig.add_trace(go.Scatter(
        x=time_axis,
        y=selected_segment[:, 0],
        mode='lines',
        name='Vibration Channel 1'
    ))
    vib_fig.add_trace(go.Scatter(
        x=time_axis,
        y=selected_segment[:, 1],
        mode='lines',
        name='Vibration Channel 2'
    ))
    vib_fig.update_layout(
        title=f"Vibration Signal for GPS Point {point_index}",
        xaxis_title="Time (s)",
        yaxis_title="Acceleration"
    )
    return vib_fig

# ====================
# Run the Dash App
# ====================
if __name__ == "__main__":
    app.run_server(debug=True, port=8060)
'''

'\nimport os\nimport pandas as pd\nimport numpy as np\nimport plotly.express as px\nimport plotly.graph_objects as go\nimport dash\nfrom dash import dcc, html, Input, Output\nimport tkinter as tk\nfrom tkinter import filedialog\n\n# ====================\n# Data Loading using Tkinter\n# ====================\nroot = tk.Tk()\nroot.withdraw()  # Hide the root Tk window\n\n# Define file keys for CSV files.\nfiles = {\n    "latitude": None,\n    "longitude": None,\n    "vibration1": None,\n    "vibration2": None,\n    "speed": None\n}\n\ndef load_file(key):\n    file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])\n    if file_path:\n        files[key] = file_path\n        print(f"{key.capitalize()} file loaded: {file_path}")\n\nprint("Select Latitude File")\nload_file("latitude")\nprint("Select Longitude File")\nload_file("longitude")\nprint("Select Vibration 1 File")\nload_file("vibration1")\nprint("Select Vibration 2 File")\nload_file("vibration2")\nprint("Select Spe